# Purpose of this file:
Author: Phuong Huong Nguyen | 29.5.2022

0. All features are calculated based on the all orders occured until **before 04.01.2021**
1. Creating a fearture named **average_cycle_days**, which tell us the average time (according to day) that one user bought a certain item in the preriod of time
- The data will be taken from **new_orders_aggregation.csv**
- Explaining the method to calculate as below:
    * For example, we know that one user A bought an item B in days_list: [23, 41, 50]:
    * --> **Average cycle = ((50 - 41) + (41 - 23)) / (len(days_list) - 1)**
    * --> To be clearer, we calculate the average distance between elements in days_list
    * days_list will be aggregated from the **new_orders_aggregation.csv**

2. Creating features **first_day_bought** and **last_day_bought**. These feature let us know when is the first time and last time one user bought a certain item

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import math

# I. Import DataFrames

In [12]:
orders_df = pd.read_csv('new_orders_aggregation.csv', sep='|', index_col = 0)
orders_df

C:\Users\dream\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,userID,itemID,date,order
0,0,1505,2020-09-01,1
1,0,6446,2020-12-11,1
2,0,6446,2021-01-15,1
3,0,9325,2020-11-20,1
4,0,12468,2020-08-03,1
...,...,...,...,...
1071015,46137,22403,2021-01-18,1
1071016,46137,22583,2021-01-31,1
1071017,46137,28343,2020-08-08,1
1071018,46137,28900,2020-08-08,2


### Import DataFrame user_history
- **user_history** is the dataframe which manage all purchase history of each userID in the period of time from June 2020 to before 4.1.2021. Therefore, from this dataframe we can search for one userID to know all the itemID they bought and the time of transaction

In [13]:
user_history = pd.read_csv('user_history.csv', index_col = 0)
user_history

,userID,itemID,date,month,timeDeltaDays,timeDeltaWeeks
0,0,1505,2020-09-01 00:00:00,9,93,14
1,0,6446,2020-12-11 00:00:00,12,194,28
2,0,9325,2020-11-20 00:00:00,11,173,25
3,0,12468,2020-08-03 00:00:00,8,64,10
4,0,12505,2020-08-18 00:00:00,8,79,12
...,...,...,...,...,...,...
918285,46137,2667,2020-09-17 00:00:00,9,109,16
918286,46137,20209,2020-08-08 00:00:00,8,69,10
918287,46137,28343,2020-08-08 00:00:00,8,69,10
918288,46137,28900,2020-08-08 00:00:00,8,69,10


In [14]:
# convert column date in df from str type to datetime type
user_history['date'] = pd.to_datetime(user_history['date'], infer_datetime_format = True, cache = True)
user_history

,userID,itemID,date,month,timeDeltaDays,timeDeltaWeeks
0,0,1505,2020-09-01,9,93,14
1,0,6446,2020-12-11,12,194,28
2,0,9325,2020-11-20,11,173,25
3,0,12468,2020-08-03,8,64,10
4,0,12505,2020-08-18,8,79,12
...,...,...,...,...,...,...
918285,46137,2667,2020-09-17,9,109,16
918286,46137,20209,2020-08-08,8,69,10
918287,46137,28343,2020-08-08,8,69,10
918288,46137,28900,2020-08-08,8,69,10


# II. Preprocessing orders_df


## 1. Calculating weeks from 1.June 2020 to 31.1 2021 in orders_df
- timeDeltaDays include the days which are counted from 1 to... , in which 1 equals to the day 01.06.2020 in column date
- timeDeltaWeeks is calculated according to timeDeltaDays. 1 week in timeDeltaWeeks includes 7 days (from 1 to 7 in timeDeltaDays)

In [15]:
date = datetime.strptime("01.06.2020", '%d.%m.%Y')
orders_df['date'] = pd.to_datetime(orders_df['date'], infer_datetime_format = True, cache = True)
orders_df['month'] = orders_df['date'].dt.month
orders_df['timeDeltaDays'] = orders_df['date'].apply(lambda x: (x-date).days + 1)
orders_df

,userID,itemID,date,order,month,timeDeltaDays
0,0,1505,2020-09-01,1,9,93
1,0,6446,2020-12-11,1,12,194
2,0,6446,2021-01-15,1,1,229
3,0,9325,2020-11-20,1,11,173
4,0,12468,2020-08-03,1,8,64
...,...,...,...,...,...,...
1071015,46137,22403,2021-01-18,1,1,232
1071016,46137,22583,2021-01-31,1,1,245
1071017,46137,28343,2020-08-08,1,8,69
1071018,46137,28900,2020-08-08,2,8,69


## 2. Removing all duplicated rows which have the same (userID, itemID, timeDeltaDays) except for the first occurrence

In [16]:
df = orders_df.copy().drop(['order'], axis=1)
df.drop_duplicates(subset=['userID', 'itemID', 'timeDeltaDays'], keep='first', inplace=True)
df


,userID,itemID,date,month,timeDeltaDays
0,0,1505,2020-09-01,9,93
1,0,6446,2020-12-11,12,194
2,0,6446,2021-01-15,1,229
3,0,9325,2020-11-20,11,173
4,0,12468,2020-08-03,8,64
...,...,...,...,...,...
1071015,46137,22403,2021-01-18,1,232
1071016,46137,22583,2021-01-31,1,245
1071017,46137,28343,2020-08-08,8,69
1071018,46137,28900,2020-08-08,8,69


- There is no combination between userID and itemID, which occurs more than once per day

## 3. Copying all data before 4.1.2021 to a new dataframe and calculating average cycle based on the new dataframe

In [17]:
df_copy = df[df['timeDeltaDays'] < 218]
df_copy

,userID,itemID,date,month,timeDeltaDays
0,0,1505,2020-09-01,9,93
1,0,6446,2020-12-11,12,194
3,0,9325,2020-11-20,11,173
4,0,12468,2020-08-03,8,64
5,0,12505,2020-08-18,8,79
...,...,...,...,...,...
1071011,46137,2667,2020-09-17,9,109
1071014,46137,20209,2020-08-08,8,69
1071017,46137,28343,2020-08-08,8,69
1071018,46137,28900,2020-08-08,8,69


## 4. Using dataframe df_copy to update all days of year, in which one user bought a certain item

In [19]:
# list_days is used to update all list of days, in which one user bought a certain item. 
list_days = []

for i in range(len(df_copy['userID'])):
    user = df_copy.iloc[i, 0]
    item = df_copy.iloc[i, 1]
    # get a sub-dataframe which match with user and item
    data = user_history[(user_history['userID'] == user) & (user_history['itemID'] == item)]
    days = data['timeDeltaDays'].to_list()
    list_days.append(days)
    

# adding list_days to the dataframe df_copy
df_copy['days_bought_item'] = list_days
df_copy

<ipython-input-19-4044053f3d22>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['days_bought_item'] = list_days


,userID,itemID,date,month,timeDeltaDays,days_bought_item
0,0,1505,2020-09-01,9,93,[93]
1,0,6446,2020-12-11,12,194,[194]
3,0,9325,2020-11-20,11,173,[173]
4,0,12468,2020-08-03,8,64,[64]
5,0,12505,2020-08-18,8,79,[79]
...,...,...,...,...,...,...
1071011,46137,2667,2020-09-17,9,109,[109]
1071014,46137,20209,2020-08-08,8,69,[69]
1071017,46137,28343,2020-08-08,8,69,[69]
1071018,46137,28900,2020-08-08,8,69,[69]


In [20]:
# extract dataframe df into a .csv file
#df_copy.to_csv('3_days_bought_item.csv')

## 5. Calculate the average cycle of purchasing of each user for each certain item ( That means how long did one user buy a certain item again). The calculation is according to day

### a. writing a function which calculate the average distance between elements in a list 
- For example: give list = [23, 41, 50]: -->average = ((50 - 41) + (41 - 23)) / (len(list) - 1)

In [21]:
def calculate_distance(list_distance):
    avg_cicle = 0
    # if list_distance has only one element (that means user bought that item only one time), so avg_cicle should be 0
    if len(list_distance) <= 1:
        return 0
    else:
        for i in range(len(list_distance) - 1, 0, -1): # i goes down to 1
            j = i - 1
            avg_cicle = avg_cicle + list_distance[i] - list_distance[j]
        return round(avg_cicle / (len(list_distance) - 1), 2)  # result is rounded to 2 decimals

### b. Applying function calculate_distance to all rows of column 'days_bought_item' in df_copy and save it into a new column named 'average_cycle_days'

In [22]:
df_copy['average_cycle_days'] = df_copy.apply(lambda row: calculate_distance(row['days_bought_item']), axis = 1)

<ipython-input-22-cff330e827aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['average_cycle_days'] = df_copy.apply(lambda row: calculate_distance(row['days_bought_item']), axis = 1)


In [23]:
df_copy

,userID,itemID,date,month,timeDeltaDays,days_bought_item,average_cycle_days
0,0,1505,2020-09-01,9,93,[93],0.0
1,0,6446,2020-12-11,12,194,[194],0.0
3,0,9325,2020-11-20,11,173,[173],0.0
4,0,12468,2020-08-03,8,64,[64],0.0
5,0,12505,2020-08-18,8,79,[79],0.0
...,...,...,...,...,...,...,...
1071011,46137,2667,2020-09-17,9,109,[109],0.0
1071014,46137,20209,2020-08-08,8,69,[69],0.0
1071017,46137,28343,2020-08-08,8,69,[69],0.0
1071018,46137,28900,2020-08-08,8,69,[69],0.0


In [24]:
# extract dataframe df into a .csv file
#df_copy.to_csv('3_average_cycle_days.csv')

## 6. Extracting the first day and the last day of occurence of combination between userID and itemID

In [25]:
extract_days = df_copy.copy().drop(columns=['month', 'timeDeltaDays', 'days_bought_item', 'average_cycle_days'])
extract_days

,userID,itemID,date
0,0,1505,2020-09-01
1,0,6446,2020-12-11
3,0,9325,2020-11-20
4,0,12468,2020-08-03
5,0,12505,2020-08-18
...,...,...,...
1071011,46137,2667,2020-09-17
1071014,46137,20209,2020-08-08
1071017,46137,28343,2020-08-08
1071018,46137,28900,2020-08-08


### a. Removing all duplicated combination between userID and itemId in extract_days except for the first (last) occurence, so that we get the first_date_bought (last_date_bought) column. Then merging these columns to the df_copy

In [54]:
keep_first_day = extract_days.drop_duplicates(subset=['userID', 'itemID'], keep = 'first')
keep_first_day.rename(columns = {'date': 'first_date_bought'}, inplace=True)
keep_first_day

C:\Users\dream\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,userID,itemID,first_date_bought
0,0,1505,2020-09-01
1,0,6446,2020-12-11
3,0,9325,2020-11-20
4,0,12468,2020-08-03
5,0,12505,2020-08-18
...,...,...,...
1071011,46137,2667,2020-09-17
1071014,46137,20209,2020-08-08
1071017,46137,28343,2020-08-08
1071018,46137,28900,2020-08-08


In [55]:
keep_last_day = extract_days.drop_duplicates(subset=['userID', 'itemID'], keep = 'last')
keep_last_day.rename(columns = {'date': 'last_date_bought'}, inplace = True)
keep_last_day

C:\Users\dream\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,userID,itemID,last_date_bought
0,0,1505,2020-09-01
1,0,6446,2020-12-11
3,0,9325,2020-11-20
4,0,12468,2020-08-03
5,0,12505,2020-08-18
...,...,...,...
1071011,46137,2667,2020-09-17
1071014,46137,20209,2020-08-08
1071017,46137,28343,2020-08-08
1071018,46137,28900,2020-08-08


### b. Merging keep_first_day and keep_last_day with df_copy

In [56]:
final_df = df_copy.merge(keep_first_day, how = 'left', on = ['userID', 'itemID'])
final_df

,userID,itemID,date,month,timeDeltaDays,days_bought_item,average_cycle_days,first_date_bought
0,0,1505,2020-09-01,9,93,[93],0.0,2020-09-01
1,0,6446,2020-12-11,12,194,[194],0.0,2020-12-11
2,0,9325,2020-11-20,11,173,[173],0.0,2020-11-20
3,0,12468,2020-08-03,8,64,[64],0.0,2020-08-03
4,0,12505,2020-08-18,8,79,[79],0.0,2020-08-18
...,...,...,...,...,...,...,...,...
919701,46137,2667,2020-09-17,9,109,[109],0.0,2020-09-17
919702,46137,20209,2020-08-08,8,69,[69],0.0,2020-08-08
919703,46137,28343,2020-08-08,8,69,[69],0.0,2020-08-08
919704,46137,28900,2020-08-08,8,69,[69],0.0,2020-08-08


In [57]:
dataset = final_df.merge(keep_last_day, how='left', on=['userID', 'itemID'])
dataset

,userID,itemID,date,month,timeDeltaDays,days_bought_item,average_cycle_days,first_date_bought,last_date_bought
0,0,1505,2020-09-01,9,93,[93],0.0,2020-09-01,2020-09-01
1,0,6446,2020-12-11,12,194,[194],0.0,2020-12-11,2020-12-11
2,0,9325,2020-11-20,11,173,[173],0.0,2020-11-20,2020-11-20
3,0,12468,2020-08-03,8,64,[64],0.0,2020-08-03,2020-08-03
4,0,12505,2020-08-18,8,79,[79],0.0,2020-08-18,2020-08-18
...,...,...,...,...,...,...,...,...,...
919701,46137,2667,2020-09-17,9,109,[109],0.0,2020-09-17,2020-09-17
919702,46137,20209,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08
919703,46137,28343,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08
919704,46137,28900,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08


### c. Getting the first day and the last day that the user bought a certain item based on 'timeDeltaDays'

In [59]:
dataset['first_day_delta_bought'] = dataset.apply(lambda row : row['days_bought_item'][0], axis=1)
dataset

,userID,itemID,date,month,timeDeltaDays,days_bought_item,average_cycle_days,first_date_bought,last_date_bought,first_day_delta_bought
0,0,1505,2020-09-01,9,93,[93],0.0,2020-09-01,2020-09-01,93
1,0,6446,2020-12-11,12,194,[194],0.0,2020-12-11,2020-12-11,194
2,0,9325,2020-11-20,11,173,[173],0.0,2020-11-20,2020-11-20,173
3,0,12468,2020-08-03,8,64,[64],0.0,2020-08-03,2020-08-03,64
4,0,12505,2020-08-18,8,79,[79],0.0,2020-08-18,2020-08-18,79
...,...,...,...,...,...,...,...,...,...,...
919701,46137,2667,2020-09-17,9,109,[109],0.0,2020-09-17,2020-09-17,109
919702,46137,20209,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08,69
919703,46137,28343,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08,69
919704,46137,28900,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08,69


In [60]:
dataset['last_day_delta_bought'] = dataset.apply(lambda row : row['days_bought_item'][-1], axis=1)
dataset

,userID,itemID,date,month,timeDeltaDays,days_bought_item,average_cycle_days,first_date_bought,last_date_bought,first_day_delta_bought,last_day_delta_bought
0,0,1505,2020-09-01,9,93,[93],0.0,2020-09-01,2020-09-01,93,93
1,0,6446,2020-12-11,12,194,[194],0.0,2020-12-11,2020-12-11,194,194
2,0,9325,2020-11-20,11,173,[173],0.0,2020-11-20,2020-11-20,173,173
3,0,12468,2020-08-03,8,64,[64],0.0,2020-08-03,2020-08-03,64,64
4,0,12505,2020-08-18,8,79,[79],0.0,2020-08-18,2020-08-18,79,79
...,...,...,...,...,...,...,...,...,...,...,...
919701,46137,2667,2020-09-17,9,109,[109],0.0,2020-09-17,2020-09-17,109,109
919702,46137,20209,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08,69,69
919703,46137,28343,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08,69,69
919704,46137,28900,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08,69,69


In [62]:
# extract dataframe df into a .csv file
#dataset.to_csv('3_dataset_before_drop_duplicates.csv')

## 7. Removing all duplicated data (userID, itemID) except for the first one, so that we can merge correctly with train_test_set when we train models

In [63]:
final_dataset = dataset.drop_duplicates(subset=['userID', 'itemID'], keep='first')
final_dataset

,userID,itemID,date,month,timeDeltaDays,days_bought_item,average_cycle_days,first_date_bought,last_date_bought,first_day_delta_bought,last_day_delta_bought
0,0,1505,2020-09-01,9,93,[93],0.0,2020-09-01,2020-09-01,93,93
1,0,6446,2020-12-11,12,194,[194],0.0,2020-12-11,2020-12-11,194,194
2,0,9325,2020-11-20,11,173,[173],0.0,2020-11-20,2020-11-20,173,173
3,0,12468,2020-08-03,8,64,[64],0.0,2020-08-03,2020-08-03,64,64
4,0,12505,2020-08-18,8,79,[79],0.0,2020-08-18,2020-08-18,79,79
...,...,...,...,...,...,...,...,...,...,...,...
919701,46137,2667,2020-09-17,9,109,[109],0.0,2020-09-17,2020-09-17,109,109
919702,46137,20209,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08,69,69
919703,46137,28343,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08,69,69
919704,46137,28900,2020-08-08,8,69,[69],0.0,2020-08-08,2020-08-08,69,69


In [64]:
# extract dataframe df into a .csv file
#final_dataset.to_csv('3_dataset_without_duplicates.csv')

In [65]:
IU_FEAT_Average_Day = final_dataset.drop(columns = ['date', 'month', 'timeDeltaDays', 'days_bought_item'])
IU_FEAT_Average_Day

,userID,itemID,average_cycle_days,first_date_bought,last_date_bought,first_day_delta_bought,last_day_delta_bought
0,0,1505,0.0,2020-09-01,2020-09-01,93,93
1,0,6446,0.0,2020-12-11,2020-12-11,194,194
2,0,9325,0.0,2020-11-20,2020-11-20,173,173
3,0,12468,0.0,2020-08-03,2020-08-03,64,64
4,0,12505,0.0,2020-08-18,2020-08-18,79,79
...,...,...,...,...,...,...,...
919701,46137,2667,0.0,2020-09-17,2020-09-17,109,109
919702,46137,20209,0.0,2020-08-08,2020-08-08,69,69
919703,46137,28343,0.0,2020-08-08,2020-08-08,69,69
919704,46137,28900,0.0,2020-08-08,2020-08-08,69,69


In [66]:
# extract dataframe into .csv
#IU_FEAT_Average_Day.to_csv('3_IU_FEAT_Average_Day.csv')

# III. Analyzing submission.csv based on all history orders until before day 04.01.2021